# III) Création d'une ontologie

Pour ce faire, nous utiliserons le package `owlready2` qui permet de créer des ontologies en python. (https://linuxfr.org/news/owlready-un-module-python-pour-manipuler-les-ontologies-owl)
Les données seront extraite de la base de données de WikiData, nous interrogerons cette base de données par l'API mis à disposition par la librairie `wikibaseintegrator` (https://github.com/LeMyst/WikibaseIntegrator#execute-sparql-queries).

Commençons par installer les packages nécessaires

In [ ]:
# !pip install owlready2
# !pip install wikibaseintegrator

Récupérons les classes et sous-classes de la classe Animalia du fichier `animals_id_name_subClass_sub2Class.csv` que nous avons créé précédemment. Nous avons décité de se limiter à deux sous-classes pour des raisons de temps de récupération des informations de l'API.
De plus, nous rencontrons un problème quand les sous classes ne sont pas définies dans WikiData, nous avons donc décidé de ne pas les prendre en compte, et de créer une sous-classe d'animalia appelé sans famille.

In [2]:
import pandas as pd

df = pd.read_csv("animals_id_name_subClass_sub2Class.csv")

tableau_animaux = df['animalLabel'].values.tolist()

# Sélectionner les colonnes spécifiques
selected_columns = ["subclass2Label", "subclassLabel"]
selected_data = df[selected_columns]

# Transformer les colonnes en un tableau de la forme [[subclass2Label], [subclassLabel]]
tableau = selected_data.values.T.tolist()

# Afficher le résultat
print(tableau)

# Afficher le résultat
print(tableau_animaux)

[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'felidae', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'penguin', 'mammal', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'felidae', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'felidae', 'felidae', 'felidae', '0', 'animal', '0', '0', '0', '0', '0', 'fish', '0', '0', '0', '0', '0', '0', 'food ingredient', '0', '0', '0', '0', '0', 'elephant', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'big cat', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'penguin', 'elephant', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'big cat', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'big cat', 'big cat', 'big cat', '0', 'bear', '0', '0', '0', '0', '0', 'electric fish', '0', '0', '0', '0', '0', '0', 'vegetable', '0', '0', '0', 

Nous devons modifier la structure des données pour qu'elle soit exploitable par la suite. Par exemple, nous avons un animal qui a pour sous-classe "éléphant" et "mammifère" un autre a pour sous-classe "éléphant d'Asie" et éléphant", donc nous devons créer une classe éléphant d'Asie qui découle d'éléphant qui elle-même découle de mammifère.

In [7]:
# On récupére les sous-sous-classes en supprimant les 0 et les doublons
import numpy as np

# Supprimer les zéros
tableau_sans_zeros = [x for x in tableau[0] if x != '0']
# Supprimer les doublons
tableau_sans_doublons = np.unique(tableau_sans_zeros)
print(tableau_sans_doublons)

# On parcourt les sous-sous-classes récupérées et on les ajoute un tableau de la forme [[..., sous-sous-sous-classe, sous-sous-classe, sous-classe], ...]
tableau_classes = [[x] for x in tableau_sans_doublons]

for i in range(len(tableau_classes)):
    for j in range(len(tableau[0])):
        if tableau[0][j] != '0' and tableau[1][j] != '0':
            if tableau_sans_doublons[i] == tableau[0][j] and tableau[1][j] not in tableau_classes[i]: # On met le 1er élément de la sous-sous-classe dans le tableau
                # On vient de trouver une sous-sous-classe, maintenant on va l'ajouter à notre tableau
                tableau_classes[i].append(tableau[1][j])
            elif tableau_sans_doublons[i] == tableau[1][j] and tableau[0][j] not in tableau_classes[i]: # On met le 2ème élément de la sous-sous-classe dans le tableau
                # On vient de trouver une sous-sous-classe, maintenant on va l'ajouter en tête à notre tableau
                tableau_classes[i].insert(0, tableau[0][j])
            else:
                pass
        else:
            pass


def is_subset(l1, l2):
    return set(l1).issubset(l2)

descendance_classe = []
for i in range(len(tableau_classes)):
    if not any(is_subset(tableau_classes[i], tableau_classes[j]) for j in range(len(tableau_classes)) if i != j):
        descendance_classe.append(tableau_classes[i])


# Afficher le résultat
print(tableau_classes)

# Afficher le résultat
print(descendance_classe)

['animal' 'elephant' 'felidae' 'fish' 'food ingredient' 'mammal' 'penguin']
[['animal', 'bear'], ['mammal', 'elephant', 'asian elephant'], ['felidae', 'big cat'], ['fish', 'electric fish'], ['food ingredient', 'vegetable'], ['mammal', 'elephant'], ['penguin']]
[['animal', 'bear'], ['mammal', 'elephant', 'asian elephant'], ['felidae', 'big cat'], ['fish', 'electric fish'], ['food ingredient', 'vegetable'], ['penguin']]


On récupère la liste des images de l'évaluation, si elle est reconnue ou non, si oui 1, sinon 0. Et on formate le fichier pour qu'il soit exploitable par la suite.

In [34]:
image_reconnue = pd.read_csv("./reconnues.csv")

image_reconnue.head()

,nom_image,reconnue,prediction
0,n01443537_8662,0,n01614925
1,n01443537_8693,1,n01443537
2,n01443537_8719,0,n01614925
3,n01443537_875,1,n01443537
4,n01443537_8787,1,n01443537


In [35]:
image_reconnue = pd.read_csv("./imagesReconnue.csv")

image_reconnue.head()

# Le transformer en tableau de la forme [[image, reconnue], ...]
tableau_image_reconnue = image_reconnue.values.tolist()

# Afficher le résultat
print(tableau_image_reconnue)

[['n10_01', 'lion', 'big_cat', 1], ['n20_02', 'lion', 'big_cat', 0]]


Concernant la contrainte de la propriété possède pour les sous-classes d'Animalia, c'est un concept avancé dans OWL appelé "Restrictions sur les propriétés". Pour répondre au mieux à cette contrainte, nous allons utiliser AllDisjoint et AllDifferent.

In [42]:
import numpy as np
from owlready2 import Thing, AllDisjoint
import owlready2 as owl

onto = owl.get_ontology("http://example651181.com/animalia.owl")

# Définir la classe racine Animalia
with onto:
    class Animalia(Thing):
        pass


# Définir les sous-classes
with onto:
    for i in range(len(tableau)):
        # Supprimer les zéros
        tableau_sans_zeros = [x for x in tableau[i] if x != '0']
        # Supprimer les doublons
        tableau_sans_doublons = np.unique(tableau_sans_zeros)
        for j in range(len(tableau_sans_doublons)):
            if i == 0: # cela dépend de Animalia
                nom_classe = tableau_sans_doublons[j].replace(" ", "_") # Remplacer les espaces par des underscores
                # Créer une classe dynamiquement
                classe = type(nom_classe, (Animalia,), {})
            else: # c'est une sous-classe du précédent
                indices_ssClass = [i for i, x in enumerate(tableau[i]) if x == tableau_sans_doublons[j]] # Récupérer les indices des sous-sous-classes
                if len(indices_ssClass) == 1: # Alors on crée sans réfléchir la sous classe
                    nom_sous_classe = tableau_sans_doublons[j].replace(" ", "_")  # Remplacer les espaces par des underscores
                    nom_classe = tableau[i-1][indices_ssClass[0]].replace(" ", "_")  # Remplacer les espaces par des underscores
                    # Créer une classe dynamiquement
                    classe = type(nom_sous_classe, (getattr(onto, nom_classe),), {})
                else: # Alors on crée une classe en vérifiant s'il y a des doublons
                    suppresion_doublons = []
                    for k in range(len(indices_ssClass)):
                        suppresion_doublons.append([indices_ssClass[k], tableau[i][indices_ssClass[k]]])
                    uniques = [sublist for i, sublist in enumerate(suppresion_doublons) if sublist[1] not in [x[1] for x in suppresion_doublons[:i]]]
                    for l in range(len(uniques)):
                        nom_sous_classe = uniques[l][1].replace(" ", "_")  # Remplacer les espaces par des underscores
                        nom_classe = tableau[i-1][uniques[l][0]].replace(" ", "_")  # Remplacer les espaces par des underscores
                        # Créer une classe dynamiquement
                        classe = type(nom_sous_classe, (getattr(onto, nom_classe),), {})

# Définir les classes orphelines
with onto:
    class SansFamille(Animalia):
        pass

# Définition de la classe image avec son nom et si elle est reconnue ou non
with onto:
    class Image(Thing):
        pass

    class Reconnue(Image >> bool):
        pass

# Définir les exclusions, toutes les classes d'un même niveau sont disjointes
with onto:
    for i in range(len(tableau)):
        # Supprimer les zéros
        tableau_sans_zeros = [x for x in tableau[i] if x != '0']
        # Supprimer les doublons
        tableau_sans_doublons = np.unique(tableau_sans_zeros)
        # Créer une liste de classes
        liste_classes = []
        for j in range(len(tableau_sans_doublons)):
            nom_classe = tableau_sans_doublons[j].replace(" ", "_")  # Remplacer les espaces par des underscores
            liste_classes.append(getattr(onto, nom_classe))
        # Créer une liste de classes disjointes
        AllDisjoint(liste_classes)

# Définir les différentes propriétés
with onto:
    class CaracteristiqueMorphologique(Thing):
        pass

    class Patte(CaracteristiqueMorphologique):
        pass

    class Museau(CaracteristiqueMorphologique):
        pass

    class Aile(CaracteristiqueMorphologique):
        pass

    class Bec(CaracteristiqueMorphologique):
        pass

    class Queue(CaracteristiqueMorphologique):
        pass

    class Poil(CaracteristiqueMorphologique):
        pass

    class Plume(CaracteristiqueMorphologique):
        pass

# Pour définir les associations, on utilise la propriété contient, qui permet de relier une image à un animal, et on lui précisera si l'image a bien été reconnue, avec la propriété possède, qui permet de relier un animal à une caractéristique morphologique.
with onto:
    class Contient(owl.ObjectProperty):
        domain = [Image]
        range = [Animalia]
        pass

    class Possede(owl.ObjectProperty):
        domain    = [Animalia]
        range     = [CaracteristiqueMorphologique]
        pass


for i in range(len(tableau[-1])):
    if tableau[-1][i] == '0':
        nom_animal = tableau[0][i].replace(" ", "_")  # Remplacer les espaces par des underscores
        animal = SansFamille(nom_animal)
    else:
        nom_classe = tableau[-1][i].replace(" ", "_")  # Remplacer les espaces par des underscores
        nom_animal = tableau_animaux[i].replace(" ", "_")  # Remplacer les espaces par des underscores
        # Accédez à la classe spécifique
        classe = onto[nom_classe]
        # Créez une nouvelle instance de cette classe
        nouvel_animal = classe(nom_animal)


# On ajoute les images à notre ontologie.
for i in range(len(tableau_image_reconnue)):
    nom_image = tableau_image_reconnue[i][0].replace(" ", "_")  # Remplacer les espaces par des underscores
    image_instance = Image(nom_image)
    if tableau_image_reconnue[i][3] == 1:
        print("L'image", nom_image, "a été reconnue")
        image_instance.Reconnue.append(True)
    else:
        image_instance.Reconnue.append(False)

    nom_classe = tableau_image_reconnue[i][1].replace(" ", "_")
    nom_sous_classe = tableau_image_reconnue[i][2].replace(" ", "_")

    print("nom_sous_classe", nom_sous_classe)

    nom_sous_classe = onto[nom_sous_classe]

    print(nom_sous_classe)

    classe_instance = onto.search_one(type=nom_sous_classe, iri="*"+nom_classe)

    print(classe_instance)
    if classe_instance is None:
        classe_instance = nom_sous_classe(nom_classe)

    image_instance.Contient.append(classe_instance)


onto.save(file="FF3_animalia.owl", format="rdfxml")

elephant-2
animal
fish
mammal
penguin-2
food_ingredient
big_cat
animalia.big_cat
penguin-1
animalia.penguin-1
elephant-1
animalia.elephant-1
big_cat
animalia.big_cat
big_cat
animalia.big_cat
big_cat
animalia.big_cat
big_cat
animalia.big_cat
bear
animalia.bear
electric_fish
animalia.electric_fish
vegetable
animalia.vegetable
asian_elephant
animalia.asian_elephant
L'image n10_01 a été reconnue
nom_sous_classe big_cat
animalia.big_cat
animalia.lion
nom_sous_classe big_cat
animalia.big_cat
animalia.lion


On enregistre notre ontologie dans un fichier .owl

In [ ]:
#onto.save(file="FF3_animalia.owl", format="rdfxml")